### **aperiodic_irasa**

Separate aperiodic activity from 1-60 Hz power spectra using the IRASA method, and compute the aperiodic slope and offset (intercept). 

IRASA described in: Wen, H, Liu, Z. Separating fractal and oscillatory components in the power spectrum of neurophysiological signal. *Brain Topography* 29 (2016). [https://doi.org/10.1007/s10548-015-0448-0](https://doi.org/10.1007/s10548-015-0448-0) 

Author: EL Johnson, PhD

Copyright (c) 2022-2025  
ZR Cross, PhD & EL Johnson, PhD

### Download the sample data file:
[https://drive.google.com/file/d/1KBlYuFC8-K31LRf22ueaTliSrNZENwnZ/view?usp=sharing](https://drive.google.com/file/d/1KBlYuFC8-K31LRf22ueaTliSrNZENwnZ/view?usp=sharing)

### Set directories:

In [ ]:
datdir = 'C:\\Users\\elizl\\Downloads\\' # replace with your path to the downloaded data
savdir = datdir # replace with your save path

### Import modules:

In [ ]:
import os
import glob

!pip install mne
import mne

!pip install yasa
import yasa

# for compatibility with MNE
!conda install -y -c conda-forge numpy=1.26.4 --force-reinstall
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

### Read sample data file:

In [ ]:
fid = 'NM01_new.mat-epo.fif.gz'
fid_pth = os.path.join(datdir, fid)
epochs = mne.read_epochs(fid_pth, preload=True)

### Extract subject ID and set task:

In [ ]:
sid = fid.split('_')[0]
task = 'WM' # WM/Scene/Rest - the sample dataset is WM
print(f'Ready to process subject {sid}.')

### Extract info from the epoched data:

In [ ]:
data = epochs.get_data(tmin = 0, tmax = 3) # 0 to 3 s after stimulus onset
sf = epochs.info['sfreq'] # set sf as the sampling frequency
chan = epochs.ch_names # define channel as ch_names

### Run IRASA:

In [ ]:
# initialize data frames
dfs = []
psd_total = []
df_aperiodic = []

# loop through trials
for idx in range(data.shape[0]):
    freqs, psd_aperiodic, psd_osc, fit_params = yasa.irasa(data[idx, :, :], sf, 
                                                               ch_names = chan, 
                                                               band = (1, 60), 
                                                               win_sec = 1, 
                                                               return_fit = True)
    
    # generate data frames required for later analysis
    fit_params.insert(loc = 0, column = 'epoch', value = idx) # add epoch column
    fit_params['subj'] = sid # add subject column
    fit_params['task'] = task # add task column
    dfs.append(fit_params) # append each epoch data frame
    
    # append the PSD arrays
    psd_total.append(psd_osc)
    
    # generate data frame for aperiodic component
    df_aperiodic_epoch = pd.DataFrame(psd_aperiodic)
    df_aperiodic_epoch.insert(loc = 0, column = 'epoch', value = idx)
    df_aperiodic_epoch['subj'] = sid # add subject column
    df_aperiodic_epoch['task'] = task # add task column
    df_aperiodic.append(df_aperiodic_epoch) # add subject column

### Save fit parameters:

In [ ]:
df = pd.concat(dfs)
df.to_csv(f'{savdir}fit_params_{sid}_{task}.csv', sep = ',', mode = 'a', header = True)
df.head() # show top of the saved table

### Save aperiodic components:

In [ ]:
df_aperiodic_df = pd.concat(df_aperiodic)
df.to_csv(f'{savdir}psd_aperiodic_{sid}_{task}.csv', sep = ',', mode = 'a', header = True)
df_aperiodic_df.head() # show top of the saved table

### Plot the aperiodic component for a sample channel:

In [ ]:
plt.plot(freqs, psd_aperiodic[2, :], 'k', lw = 2.5)
#plt.fill_between(freqs, psd_aperiodic[2, :], cmap = 'Spectral')
plt.xlim(1, 60)
plt.yscale('log') # log-log space
plt.title('Aperiodic component at ' + chan[0], fontsize = 15)
plt.xlabel('Frequency [Hz]',fontsize = 15)
plt.ylabel('PSD log [$uV^2$/Hz]',fontsize = 15)